In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


real_final_df = pd.DataFrame(columns=["질병명", "식사요법의 필요성", "식사요법의 실제", "권장 식품", "주의 식품","그 외 주의사항"])

for i in range(11):
    # 질병 num 설정
    i = i+1
    page = str(i)

    # 기본 URL 설정
    page_url = "https://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyList.do?pageIndex="+page+"&searchCondition=&searchKeyword=&mtId=19"

    # 페이지 요청 및 HTML 파싱
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    select_list = [
        '#listForm > div > div > ul > li.first > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(2) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(3) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(4) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(5) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(6) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(7) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(8) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(9) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(10) > dl > dt > a'
    ]

    def extract_section(soup, title):
        dt_tag = soup.find("dt", string=title)
        if dt_tag:
            dd_content = dt_tag.find_next_sibling("dd").get_text(strip=True)
            return dd_content
        return "N/A"

    # 빈 DataFrame 생성
    final_df = pd.DataFrame(columns=["질병명", "식사요법의 필요성", "식사요법의 실제", "권장 식품", "주의 식품","그 외 주의사항"])

    for i in select_list:
        
        disease_name = soup.select(i)
        link_html = disease_name[0]
        link = link_html.attrs['href']
        disease_name_text = disease_name[0].text.strip()

        # 각 항목에 대한 내용 크롤링
        disease_name_url = "https://www.amc.seoul.kr"+link

        response = requests.get(disease_name_url)
        soup2 = BeautifulSoup(response.text, 'html.parser')

        # "식사요법의 필요성", "식사요법의 실제", "권장 식품", "주의 식품" 크롤링
        necessity = extract_section(soup2, "식사요법의 필요성")
        actual_therapy = extract_section(soup2, "식사요법의 실제")
        recommended_foods = extract_section(soup2, "권장 식품")
        caution_foods = extract_section(soup2, "주의 식품")
        etc = extract_section(soup2, "그 외 주의사항")

        # DataFrame 생성
        data = {
            "질병명" : [disease_name_text],
            "식사요법의 필요성": [necessity],
            "식사요법의 실제": [actual_therapy],
            "권장 식품": [recommended_foods],
            "주의 식품": [caution_foods],
            "그 외 주의사항": [etc]
        }

        df = pd.DataFrame(data)

        # DataFrame을 최종 DataFrame에 추가
        final_df = pd.concat([final_df, df], ignore_index=True)

    # 최종 DataFrame 출력
    real_final_df = pd.concat([final_df, real_final_df], ignore_index=True)
    real_final_df

IndexError: list index out of range

In [53]:
real_final_df

,질병명,식사요법의 필요성,식사요법의 실제,권장 식품,주의 식품,그 외 주의사항
0,구루병식,비타민 D 섭취 부족으로 생기는 질병으로 정상적인 칼슘의 축적이 부족하여 골 형성 ...,"1. 비타민 D가 많은 식품을 섭취합니다. (등푸른 생선, 연어, 쥐치, 가자미, ...","우유, 요구르트, 치즈, 두부, 새우, 정어리, 연어, 녹색콩류, 시금치, 브로콜리...","카페인(커피, 콜라, 초콜릿), 주류(소주, 맥주, 막걸리 등)","유당 불내증이 있거나, 유제품을 전혀 안먹거나, 완전 채식만 하는 사람들은 반드시 ..."
1,골다공증식,골다공증 환자와 골다공증으로 진행될 위험이 높은 환자에게 식사 중 칼슘 섭취량을 ...,1. 균형있는 식사를 합니다.2. 칼슘이 많은 식품을 먹습니다.3. 과도한 알코올 ...,"- 우유 및 유제품 (발효유, 치이즈, 아이스크림)- 두류 (콩, 두부 등) 및 해...","- 카페인음료(커피, 홍차등), 탄산음료(사이다, 콜라등)- 술(하루 소주 3잔이상...","칼슘 보충제는 우유나 요쿠르트와 함께 복용하는 것이 좋으며, 식사와 식사 사이에 복..."
2,고혈압식 / 폐성 고혈압식,"체중 조절을 위해 저지방, 저칼로리식을 고려하고, 염분을 제한함으로써 부종과 복수가...","1. 밥은 현미, 보리 등을 섞은 잡곡밥이 좋으며 기호에 따라 적정량 혼식을 합니다...","신선한 채소를 비롯한 잡곡, 어육류, 생선, 두부, 견과류 그리고 제철 과일과 우유...","염장 식품 : 김치류, 젓갈류, 장아찌류, 건어물 및 자반 생선류 등가공식품 : 햄...",N/A
3,고칼슘혈증식,혈중 칼슘치를 정상으로 유지하고 소변으로 나오는 칼슘량을 최소한으로 줄이기 위하여 ...,1. 편식하지 않고 골고루 섭취합니다2. 칼슘이 많이 함유된 식품을 제한합니다.3....,"흰밥, 국수, 식빵, 감자쇠고기, 돼지고기, 닭고기, 계란, 전복, 홍합호박,무우,...","녹두, 율무, 팥뼈째먹는 생선(잔멸피, 뱅어포, 양미리, 미꾸라지 등) 고등어 통조...",수유부나 폐경기 여성에게서는 칼슘을 제한하지 않고 1200~1500mg 섭취합니다....
4,고지혈증식,"포화지방산과 콜레스테롤의 과잉 섭취, 칼로리 섭취와 소모의 불균형, 당질의 과잉 섭...","1. 밥은 현미, 보리 등을 섞은 잡곡밥이 좋으며 기호에 따라 적정량 혼식을 합니다...","신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 ...","고지방 어육류 : 갈비, 삼겹살, 육류의 껍질과 기름, 장어, 햄류 등고콜레스테롤 ...",N/A
...,...,...,...,...,...,...
95,허혈성심장질환식,심장질환 시 식사요법은 크게 3가지로 나누어 생각할 수 있습니다.[콜레스테롤에 대한...,"1. 밥은 현미, 보리 등을 섞은 잡곡밥이 좋으며 기호에 따라 적정량 혼식합니다. ...","신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 ...","고지방 어육류 : 갈비, 삼겹살, 육류의 껍질과 기름, 장어, 햄류고콜레스테롤 식품...","오메가-3 지방산은 에이코사펜타엔산(EPA), 도코사헥사엔산(DHA) 등으로 주로 ..."
96,항문암식,"수술 직후 회복을 위한 충분한 열량과 단백질을 섭취하도록 하며, 수술 후 약 6주 ...","1. 고섬유질 식품 섭취를 줄입니다. (잡곡류, 채소 및 해조류, 과일류, 견과류 ...","수술직후 : 쌀밥, 흰국수, 생선, 계란, 두부, 육류(결체조직 및 껍질 제외), 유제품","수술직후 : 잡곡, 질긴 채소, 건조 과일과 껍질, 씨앗, 견과류, 자극성 식품","1. 수술 직후 약 6주간 고섬유식품 섭취를 줄이도록 합니다.2. 수술 부위, 장루..."
97,"프로피온산혈증, 프로피온산증","발린, 이소로이신, 메치오닌, 스레오닌과 같은 아미노산의 대사에 관여하는 propi...","1. 루이신, 이소루이신, 발린, 메티오닌, 트레오닌 등 프로피온산의 전구물질이 될...","쌀, 고구마, 감자 등 탄수화물 식품과 식물성 기름, 버터류, 모든 채소와 과일류","쇠고기, 돼지고기, 닭고기, 생선 및 가공품, 우유 및 유제품 등 발린 함량이 높은...","여러가지 감염, 예방접종, 외상, 마취 및 수술과 같은 상황이 발생하면 단백질 섭취..."
98,프라더 윌리 증후군식,비만으로 인한 합병증을 조절하기 위해 필요합니다.,1. 보통의 비만아보다 더 강하게 열량을 제한합니다.2. 당뇨가 동반된 경우 당뇨 ...,"잡곡밥, 신선한 채소와 과일(당뇨가 있는 경우는 별도의 양 조절 필요), 해조류, ...","사탕이나 초컬렛, 청량음료, 단 빵류 (케익 포함) 와 같이 단순당 또는 단순당이 ...",기름지지 않은 음식이라 하더라도 식욕만큼 배를 채우게 되면 체중이 쉽게 늘어나므로 ...


In [54]:
    # 기본 URL 설정
    page_url = "https://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyList.do?pageIndex=11&searchCondition=&searchKeyword=&mtId=19"

    # 페이지 요청 및 HTML 파싱
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    select_list = [
        '#listForm > div > div > ul > li.first > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(2) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(3) > dl > dt > a',
        '#listForm > div > div > ul > li:nth-child(4) > dl > dt > a'
    ]

    def extract_section(soup, title):
        dt_tag = soup.find("dt", string=title)
        if dt_tag:
            dd_content = dt_tag.find_next_sibling("dd").get_text(strip=True)
            return dd_content
        return "N/A"

    # 빈 DataFrame 생성
    final_df = pd.DataFrame(columns=["질병명", "식사요법의 필요성", "식사요법의 실제", "권장 식품", "주의 식품","그 외 주의사항"])

    for i in select_list:
        
        disease_name = soup.select(i)
        link_html = disease_name[0]
        link = link_html.attrs['href']
        disease_name_text = disease_name[0].text.strip()

        # 각 항목에 대한 내용 크롤링
        disease_name_url = "https://www.amc.seoul.kr"+link

        response = requests.get(disease_name_url)
        soup2 = BeautifulSoup(response.text, 'html.parser')

        # "식사요법의 필요성", "식사요법의 실제", "권장 식품", "주의 식품" 크롤링
        necessity = extract_section(soup2, "식사요법의 필요성")
        actual_therapy = extract_section(soup2, "식사요법의 실제")
        recommended_foods = extract_section(soup2, "권장 식품")
        caution_foods = extract_section(soup2, "주의 식품")
        etc = extract_section(soup2, "그 외 주의사항")

        # DataFrame 생성
        data = {
            "질병명" : [disease_name_text],
            "식사요법의 필요성": [necessity],
            "식사요법의 실제": [actual_therapy],
            "권장 식품": [recommended_foods],
            "주의 식품": [caution_foods],
            "그 외 주의사항": [etc]
        }

        df = pd.DataFrame(data)

        # DataFrame을 최종 DataFrame에 추가
        real_final_df = pd.concat([real_final_df, df], ignore_index=True)

In [57]:
# 엑셀 파일로 저장
file_path = "real_final_df.xlsx"
real_final_df.to_excel(file_path, index=False)

In [56]:
real_final_df

,질병명,식사요법의 필요성,식사요법의 실제,권장 식품,주의 식품,그 외 주의사항
0,구루병식,비타민 D 섭취 부족으로 생기는 질병으로 정상적인 칼슘의 축적이 부족하여 골 형성 ...,"1. 비타민 D가 많은 식품을 섭취합니다. (등푸른 생선, 연어, 쥐치, 가자미, ...","우유, 요구르트, 치즈, 두부, 새우, 정어리, 연어, 녹색콩류, 시금치, 브로콜리...","카페인(커피, 콜라, 초콜릿), 주류(소주, 맥주, 막걸리 등)","유당 불내증이 있거나, 유제품을 전혀 안먹거나, 완전 채식만 하는 사람들은 반드시 ..."
1,골다공증식,골다공증 환자와 골다공증으로 진행될 위험이 높은 환자에게 식사 중 칼슘 섭취량을 ...,1. 균형있는 식사를 합니다.2. 칼슘이 많은 식품을 먹습니다.3. 과도한 알코올 ...,"- 우유 및 유제품 (발효유, 치이즈, 아이스크림)- 두류 (콩, 두부 등) 및 해...","- 카페인음료(커피, 홍차등), 탄산음료(사이다, 콜라등)- 술(하루 소주 3잔이상...","칼슘 보충제는 우유나 요쿠르트와 함께 복용하는 것이 좋으며, 식사와 식사 사이에 복..."
2,고혈압식 / 폐성 고혈압식,"체중 조절을 위해 저지방, 저칼로리식을 고려하고, 염분을 제한함으로써 부종과 복수가...","1. 밥은 현미, 보리 등을 섞은 잡곡밥이 좋으며 기호에 따라 적정량 혼식을 합니다...","신선한 채소를 비롯한 잡곡, 어육류, 생선, 두부, 견과류 그리고 제철 과일과 우유...","염장 식품 : 김치류, 젓갈류, 장아찌류, 건어물 및 자반 생선류 등가공식품 : 햄...",N/A
3,고칼슘혈증식,혈중 칼슘치를 정상으로 유지하고 소변으로 나오는 칼슘량을 최소한으로 줄이기 위하여 ...,1. 편식하지 않고 골고루 섭취합니다2. 칼슘이 많이 함유된 식품을 제한합니다.3....,"흰밥, 국수, 식빵, 감자쇠고기, 돼지고기, 닭고기, 계란, 전복, 홍합호박,무우,...","녹두, 율무, 팥뼈째먹는 생선(잔멸피, 뱅어포, 양미리, 미꾸라지 등) 고등어 통조...",수유부나 폐경기 여성에게서는 칼슘을 제한하지 않고 1200~1500mg 섭취합니다....
4,고지혈증식,"포화지방산과 콜레스테롤의 과잉 섭취, 칼로리 섭취와 소모의 불균형, 당질의 과잉 섭...","1. 밥은 현미, 보리 등을 섞은 잡곡밥이 좋으며 기호에 따라 적정량 혼식을 합니다...","신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 ...","고지방 어육류 : 갈비, 삼겹살, 육류의 껍질과 기름, 장어, 햄류 등고콜레스테롤 ...",N/A
...,...,...,...,...,...,...
99,폐렴식,증상의 진행 중 식사량 감소로 인한 체중감소와 체조직 소모를 방지하기 위하여 충분한...,1. 과식보다는 간식과 함께 소량씩 자주 섭취합니다.2. 매끼 양질의 어육류나 두부...,"부드러운 살코기, 생선, 요구르트, 다양한 색상의 과일이나 채소","튀김, 육류의 기름",잦은 가래 배출로 몸에 수분이 부족하기 쉬우므로 수시로 과일쥬스나 물 등으로 보충하...
100,간이식후식,간이식을 받은 환자에게 수술 후 생길 수 있는 부작용과 합병증을 최소화하기 위해 충...,"1. 이식 초기에는 회복을 위해 충분한 에너지, 단백질 반찬을 섭취합니다.2. 섭취...","이식 초기 : 신선한 육류(살코기), 생선, 두부, 콩, 우유, 두유, 영양보충음료...","이식 후 안정기 : 라면, 자장면, 삼겹살, 베이컨, 장아찌, 젓갈, 생선알, 케이...",고칼륨혈증 발생 시 일시적으로 칼륨 함량 높은 음식의 제한이 필요할 수 있습니다.
101,간성혼수식,"고단백 식사, 변비, 탈수 등의 원인으로 간성 혼수가 유발될 수 있으므로 환자의 컨...","1. 고기, 생선, 두부, 계란, 콩 등의 단백질 섭취량을 줄입니다. 혼수 정도에 ...","밥, 떡, 감자, 고구마 등의 곡류 음식, 과일, 야채","고기, 생선, 두부, 콩, 계란, 유제품 등의 고단백 식품",녹즙 등 고농축 음식이나 민간요법은 간기능을 약화시킬 수 있으므로 주의합니다.
102,간경화식,"간경화 시 식사량 감소, 소화 및 흡수 기능 저하, 영양소 대사 변화, 영양 요구량...",1. 회복을 위해 충분한 칼로리 섭취가 필요하므로 당질과 지방을 통해 칼로리 섭취를...,"밥, 떡, 감자, 고구마 등 곡류 음식, 과일, 야채, 단백질 식품(살코기, 생선,...","절인 음식(김치, 젓갈, 장아찌, 자반류, 마른 반찬, 햄, 소시지 등), 국 국물...",녹즙 등 고농축 음식이나 민간요법은 간기능을 약화시킬 수 있으므로 주의합니다.
